In [1]:
from pyspark.sql import SparkSession
import pandas as pd 


# Create a local SparkSession
spark = SparkSession.builder \
    .appName("LocalSparkExample") \
    .master("local[*]") \
    .getOrCreate()


x = pd.read_csv("backend/code_execution/data/" + 'academic.csv')


# Create a simple DataFrame
#data = [("Alice", 1), ("Bob", 2), ("Charlie", 3)]
df = spark.createDataFrame(x)

# Show the DataFrame
df.show()

# Perform a simple transformation
result = df.select("Name").where(df.Age > 1)
result.show()

# Stop the SparkSession
spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/07 02:12:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------+--------+-----------+-------------+--------+----------+---+
|   year|students|us_students|undergraduate|graduate|non_degree|opt|
+-------+--------+-----------+-------------+--------+----------+---+
|1948/49|   25464|  2403400.0|          NaN|     NaN|       NaN|NaN|
|1949/50|   26433|  2445000.0|          NaN|     NaN|       NaN|NaN|
|1950/51|   29813|  2281000.0|          NaN|     NaN|       NaN|NaN|
|1951/52|   30462|  2102000.0|          NaN|     NaN|       NaN|NaN|
|1952/53|   33675|  2134000.0|          NaN|     NaN|       NaN|NaN|
|1953/54|   33833|  2231000.0|          NaN|     NaN|       NaN|NaN|
|1954/55|   34232|  2447000.0|      19101.0| 12118.0|       NaN|NaN|
|1955/56|   36494|  2653000.0|          NaN|     NaN|       NaN|NaN|
|1956/57|   40666|  2918000.0|          NaN|     NaN|       NaN|NaN|
|1957/58|   43391|  3324000.0|          NaN|     NaN|       NaN|NaN|
|1958/59|   47245|        NaN|          NaN|     NaN|       NaN|NaN|
|1959/60|   48486|  3640000.0|    

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Name` cannot be resolved. Did you mean one of the following? [`opt`, `year`, `graduate`, `students`, `non_degree`].;
'Project ['Name]
+- LogicalRDD [year#0, students#1L, us_students#2, undergraduate#3, graduate#4, non_degree#5, opt#6], false


In [5]:
import os 

os.listdir("backend/code_execution/data/")

['academic.csv',
 'us_videos.csv',
 'historical_temperatures.csv',
 'student_performance.csv',
 'pytest_expected_output.csv',
 'status.csv',
 'venture_funding_deals.parquet',
 'venture_funding_deals_partitioned',
 'temperatures.csv',
 'customer_shopping_data.csv',
 'venture_funding_deals_delta',
 'README.md',
 'world_population.jsonl',
 'pokemon.csv',
 'ga_20170801.json',
 'titanic_dataset.csv',
 'contoso_sales.csv',
 'my_iceberg_catalog',
 'google_store_reviews.csv',
 'toronto_weather.csv',
 'academic_detail.csv',
 'world_population.json',
 'customer_shopping_data_no_header.csv',
 'world_population.avro',
 'covid_19_deaths.csv']

In [10]:
import io

import pandas as pd
import polars as pl
from contextlib import redirect_stderr, redirect_stdout


def get_pyspark_code(code, dfs):
    session = 'from pyspark.sql import SparkSession\nspark = SparkSession.builder.appName("LocalSparkExample").master("local[*]").getOrCreate()\n'
    for df_name in dfs:
        session += f"{df_name} = spark.createDataFrame({df_name})\n"

    session += code
    session += "\nresult = result.toPandas()\nspark.stop()"
    return session


def run_code(code, datasets, preprocessing_code):
    """Execute code and return result

    Args:
        code (string): code to execute
        datasets (list[str]): list of csv datasets to load in
        preprocessing_code (string): any initial code to execute

    Returns:
        pd.DataFrame, str : Resulting dataframe and error (if error raised)
    """

    # Loading datasets in as a dictionary
    dfs = {}
    for dataset in datasets:
        x = pd.read_csv("backend/code_execution/data/" + dataset)
        dfs[dataset.replace(".csv", "")] = x


    pyspark_code = get_pyspark_code(code, dfs)    

    output = io.StringIO()
    error = io.StringIO()
    
    print(pyspark_code)

    try:
        with redirect_stdout(output), redirect_stderr(error):
            local_vars = dfs
            exec(pyspark_code, {}, local_vars)
                

        if "result" in local_vars:
            result = local_vars["result"]
            # Convert result to DataFrame if it's a Series
            if isinstance(result, pl.Series):
                result = result.to_frame()
            result = result.to_dict()
            return pd.DataFrame(result), None
        return pd.DataFrame(), None
    except Exception as e:
        print(e)



run_code('result = academic.select("students").where(academic.students > 1)', ['academic.csv'], "")


from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LocalSparkExample").master("local[*]").getOrCreate()
academic = spark.createDataFrame(academic)
result = academic.select("students").where(academic.students > 1)
result = result.toPandas()
spark.stop()


(    students
 0      25464
 1      26433
 2      29813
 3      30462
 4      33675
 ..       ...
 70   1095299
 71   1075496
 72    914095
 73    948519
 74   1057188
 
 [75 rows x 1 columns],
 None)